#### TUTORIAL RAG con LangChain: 
###### Fuente: https://python.langchain.com/v0.2/docs/tutorials/rag/

In [57]:
import os
from dotenv import load_dotenv
import cohere

load_dotenv()

api_key = os.getenv("COHERE_API_KEY")
if not api_key:
    raise ValueError("Error con las variables de entorno.")

cohere.api_key = api_key

from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r-plus")

##### 1er paso: WEB SCRAPING

- pedir la información al servidor
- parsear el html o lo que recibimos
- procesar y guardar

###### Document: es un objecto con algún page_content (str) y metadata (dict).

WebBaseLoader: uses urllib to load HTML from web URLs.

###### BeatifulSoap parsea html a text: https://www.crummy.com/software/BeautifulSoup/bs4/doc/ https://github.com/GEJ1/web_scraping_freecodecamp/blob/main/web_scraping_freeCodeCamp.pdf

In [100]:
from bs4 import BeautifulSoup
import requests

# 1. Obtener el HTML
URL_BASE = 'https://scrapepark.org/courses/spanish/'
URL_BASE2 = 'https://lilianweng.github.io/posts/2023-06-23-agent/'
pedido_obtenido = requests.get(URL_BASE)
pedido_obtenido2 = requests.get(URL_BASE2)
html_obtenido = pedido_obtenido.text
html_obtenido2 = pedido_obtenido.text

# 2. "Parsear" ese HTML
soup = BeautifulSoup(html_obtenido, "html.parser")
soup2 = BeautifulSoup(html_obtenido2, "html.parser")

In [102]:
import re

## enchulado: quita espacios en blanco
document_text = soup.get_text(strip=True)
document_text2 = soup2.get_text(strip=True)

# Strings to search for
strings_a_buscar = ["MENÚ", "©", "carpincho", "Patineta"]
strings_a_buscar2 = ["AI", "be", "number", "two"]

# Results list
resultados = []
resultados2 = []

# Search for each string in the document text
for string in strings_a_buscar:
    try:
        if re.search(string, document_text):
            resultados.append(string)
        else:
            print(f"'{string}' not found in the document.")
    except AttributeError:
        print(f"El string '{string}' no fue encontrado")

print("Resultados:")
for resultado in resultados:
    print(resultado)


for string2 in strings_a_buscar2:
    try:
        if re.search(string2, document_text2):
            resultados2.append(string2)
        else:
            print(f"'{string2}' not found in the document.")
    except AttributeError:
        print(f"El string '{string2}' no fue encontrado")

# Print final results
print("Resultados:")
for resultado2 in resultados2:
    print(resultado2)

'carpincho' not found in the document.
Resultados:
MENÚ
©
Patineta
'number' not found in the document.
'two' not found in the document.
Resultados:
AI
be


#### Almacenamiento
##### Pasos para convertir a diccionario

In [103]:
resultados.insert(0,"resultados")
resultados2.insert(0,"resultados2")

In [104]:
datos = dict(zip(resultados,resultados2))

In [105]:
datos.items()

dict_items([('resultados', 'resultados2'), ('MENÚ', 'AI'), ('©', 'be')])

In [110]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Se trae algunos atributos, los que coinciden con ese nombre de clase
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)


# Load the documents
docs = loader.load()

# Check the length of the page content
print(len(docs[0].page_content))

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600, chunk_overlap=50, add_start_index=True
)

# Split the documents
all_splits = text_splitter.split_documents(docs)



# Print the content of each chunk from index 40 to 41
for chunk in all_splits[40:41]:
    print(chunk.page_content + "\n")

# Print the length of all_splits
print(len(all_splits))

43131
Fig. 9. Comparison of MIPS algorithms, measured in recall@10. (Image source: Google Blog, 2020)
Check more MIPS algorithms and performance comparison in ann-benchmarks.com.
Component Three: Tool Use#
Tool use is a remarkable and distinguishing characteristic of human beings. We create, modify and utilize external objects to do things that go beyond our physical and cognitive limits. Equipping LLMs with external tools can significantly extend the model capabilities.

107


In [60]:
# type(all_splits)

list

DocumentLoader: Object that loads data from a source as list of Documents.

#### Siguiente paso: chunkear

###### We set add_start_index=True so that the character index at which each split Document starts within the initial Document is preserved as metadata attribute “start_index”.

In [50]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=600, chunk_overlap=50, add_start_index=True
# )
# all_splits = text_splitter.split_documents(docs)
# for chunk in all_splits[40:55]:
#     print(chunk.text + "\n")
# print(all_splits)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 8}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 124}, page_content='Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 617}, page_content='Planning\n\nS

In [39]:
# len(all_splits[0].page_content)

113

In [40]:
# all_splits[10].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'start_index': 3388}

Imprimir un extracto

In [41]:
# naive_chunks = text_splitter.split_text(docs)
# for chunk in naive_chunks[40:55]:
#   print(chunk + "\n")

TypeError: expected string or bytes-like object, got 'list'

INDEXING: Store

In [ ]:
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma

import os
from dotenv import load_dotenv

load_dotenv()

COHERE_API_KEY = os.getenv("COHERE_API_KEY")

cohere_embeddings = CohereEmbeddings(user_agent=COHERE_API_KEY, model="embed-english-light-v3.0")
##split_texts = [split.page_content for split in all_splits]

query_result = cohere_embeddings.embed_documents(all_splits)

# vectorstore = Chroma.from_documents(documents=query_result, cohere_embeddings=cohere_embeddings)

# print(vectorstore)

Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised BadRequestError: status_code: 400, body: {'message': 'invalid type: parameter texts is of type object but should be of type string. For proper usage, please refer to https://docs.cohere.com/embed-reference'}.
Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised BadRequestError: status_code: 400, body: {'message': 'invalid type: parameter texts is of type object but should be of type string. For proper usage, please refer to https://docs.cohere.com/embed-reference'}.


BadRequestError: status_code: 400, body: {'message': 'invalid type: parameter texts is of type object but should be of type string. For proper usage, please refer to https://docs.cohere.com/embed-reference'}